In [1]:
import numpy as np # linear algebra
import scipy as scipy
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
import dill

import keras as k
import cv2
from tqdm import tqdm


Using Theano backend.


In [2]:
with open('../tiffs.pkl', 'rb') as in_strm:
    all_info = dill.load(in_strm)
x_train = all_info[0]
x_test = all_info[1]
y_train = all_info[2]
y_test = all_info[3]

In [3]:
print(x_train.shape)
print(x_test.shape)

(35000, 32, 32, 4)
(5479, 32, 32, 4)


In [6]:
#creates unet block attached to previous_layer
#returns last layer of unet block
def create_unet_block(depth, width, drop_constant, act, previous_layer):
    prev = previous_layer
    toConcatLater = list()
    
    for cur_depth in range(depth):
        if cur_depth != 0:
            prev = MaxPooling2D(pool_size=(2, 2))(prev)
        conv_width = int(width * math.pow(2, cur_depth))
        conv1 = Conv2D(conv_width, (3, 3), activation=act, padding='same')(prev)
        conv2 = Conv2D(conv_width, (3, 3), activation=act, padding='same')(conv1)
        prev = conv2
        toConcatLater.append(conv2)
    prev = UpSampling2D(size=(2, 2))(prev)
    for cur_depth in reversed(range(depth - 1)):
        merged = concatenate([prev, toConcatLater[cur_depth]], axis=3)
        drop = Dropout(drop_constant)(merged)
        conv_width = int(width * math.pow(2, cur_depth))
        conv1 = Conv2D(conv_width, (3, 3), activation=act, padding='same')(drop)
        conv2 = Conv2D(conv_width, (3, 3), activation=act, padding='same')(conv1)
        if cur_depth != 0:
            up = UpSampling2D(size=(2, 2))(conv2)
            prev = up
        else:
            prev = conv2
    return prev

In [12]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Input, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras.layers.merge import concatenate
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
import math

#baseline part
input_layer = Input(shape=(32, 32, 3))
unet_1 = create_unet_block(5, 32, 0.4, 'relu', input_layer)
unet_2 = create_unet_block(5, 32, 0.4, 'relu', unet_1)
unet_3 = create_unet_block(5, 32, 0.4, 'relu', unet_2)
flat = Flatten()(unet_3)
dense1 = Dense(128, activation='relu')(flat)
dense = Dense(17, activation='sigmoid')(dense1)
model = Model(inputs=[input_layer], outputs=dense)

model.compile(loss='binary_crossentropy', # We NEED binary here, since categorical_crossentropy l1 norms the output before calculating loss.
                    optimizer='adam',
                    metrics=['accuracy'])
model.summary()


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_7 (InputLayer)             (None, 32, 32, 3)     0                                            
____________________________________________________________________________________________________
conv2d_67 (Conv2D)               (None, 32, 32, 32)    896         input_7[0][0]                    
____________________________________________________________________________________________________
conv2d_68 (Conv2D)               (None, 32, 32, 32)    9248        conv2d_67[0][0]                  
____________________________________________________________________________________________________
max_pooling2d_15 (MaxPooling2D)  (None, 16, 16, 32)    0           conv2d_68[0][0]                  
___________________________________________________________________________________________

conv2d_91 (Conv2D)               (None, 4, 4, 256)     295168      max_pooling2d_21[0][0]           
____________________________________________________________________________________________________
conv2d_92 (Conv2D)               (None, 4, 4, 256)     590080      conv2d_91[0][0]                  
____________________________________________________________________________________________________
max_pooling2d_22 (MaxPooling2D)  (None, 2, 2, 256)     0           conv2d_92[0][0]                  
____________________________________________________________________________________________________
conv2d_93 (Conv2D)               (None, 2, 2, 512)     1180160     max_pooling2d_22[0][0]           
____________________________________________________________________________________________________
conv2d_94 (Conv2D)               (None, 2, 2, 512)     2359808     conv2d_93[0][0]                  
___________________________________________________________________________________________

conv2d_116 (Conv2D)              (None, 8, 8, 128)     147584      conv2d_115[0][0]                 
____________________________________________________________________________________________________
up_sampling2d_25 (UpSampling2D)  (None, 16, 16, 128)   0           conv2d_116[0][0]                 
____________________________________________________________________________________________________
concatenate_25 (Concatenate)     (None, 16, 16, 192)   0           up_sampling2d_25[0][0]           
                                                                   conv2d_106[0][0]                 
____________________________________________________________________________________________________
dropout_25 (Dropout)             (None, 16, 16, 192)   0           concatenate_25[0][0]             
____________________________________________________________________________________________________
conv2d_117 (Conv2D)              (None, 16, 16, 64)    110656      dropout_25[0][0]        

In [18]:
x_train = x_train[:,:,:,:3]
x_test = x_test[:,:,:,:3]
print(x_train.shape)
print(x_test.shape)

(35000, 32, 32, 3)
(5479, 32, 32, 3)


In [19]:
model.fit(x_train, y_train, batch_size=128, epochs=8, verbose=1,
         validation_data=(x_test, y_test))

from sklearn.metrics import fbeta_score

p_valid = model.predict(x_test, batch_size=128)
print(y_test)
print(p_valid)
print(fbeta_score(y_test, np.array(p_valid) > 0.2, beta=2, average='samples'))

Train on 35000 samples, validate on 5479 samples
Epoch 1/8
35000/35000 [==============================] - 6934s - loss: 0.2565 - acc: 0.9014 - val_loss: 0.2322 - val_acc: 0.9080
Epoch 2/8
35000/35000 [==============================] - 6911s - loss: 0.2084 - acc: 0.9141 - val_loss: 0.1957 - val_acc: 0.9230
Epoch 3/8
35000/35000 [==============================] - 7168s - loss: 0.1845 - acc: 0.9254 - val_loss: 0.1879 - val_acc: 0.9255
Epoch 4/8
35000/35000 [==============================] - 7263s - loss: 0.1720 - acc: 0.9306 - val_loss: 0.2039 - val_acc: 0.9258
Epoch 5/8
35000/35000 [==============================] - 7270s - loss: 0.1641 - acc: 0.9345 - val_loss: 0.1876 - val_acc: 0.9280
Epoch 6/8
35000/35000 [==============================] - 7268s - loss: 0.1567 - acc: 0.9379 - val_loss: 0.1558 - val_acc: 0.9388
Epoch 7/8
35000/35000 [==============================] - 7348s - loss: 0.1523 - acc: 0.9398 - val_loss: 0.1577 - val_acc: 0.9397
Epoch 8/8
35000/35000 [=========================

In [20]:
model.fit(x_train, y_train, batch_size=128, epochs=2, verbose=1,
         validation_data=(x_test, y_test))


Train on 35000 samples, validate on 5479 samples
Epoch 1/2
35000/35000 [==============================] - 8421s - loss: 0.1454 - acc: 0.9427 - val_loss: 0.1623 - val_acc: 0.9375
Epoch 2/2
35000/35000 [==============================] - 8253s - loss: 0.1429 - acc: 0.9437 - val_loss: 0.1512 - val_acc: 0.9405


In [21]:
p_valid = model.predict(x_test, batch_size=128)
print(y_test)
print(p_valid)
print(fbeta_score(y_test, np.array(p_valid) > 0.2, beta=2, average='samples'))

[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 1 0]
 [0 0 1 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 1 0]
 [0 0 0 ..., 0 0 0]]
[[  1.08586431e-01   8.43352824e-03   5.15118480e-01 ...,   2.65355501e-02
    4.29810435e-01   4.93608899e-02]
 [  1.68982044e-01   7.47520523e-03   2.18811959e-01 ...,   1.40295774e-02
    4.67733026e-01   6.51371405e-02]
 [  2.57726520e-01   2.70673558e-02   6.03018343e-01 ...,   8.31484969e-04
    5.76483846e-01   2.86559407e-02]
 ..., 
 [  2.64241695e-01   7.83529686e-05   8.02555829e-02 ...,   4.45780788e-05
    8.08773577e-01   4.59268056e-02]
 [  3.16118419e-01   5.15623469e-05   7.51943365e-02 ...,   2.27669279e-05
    8.20782900e-01   5.53337149e-02]
 [  1.16343111e-01   3.03348270e-03   2.20453009e-01 ...,   6.16263971e-03
    2.07550749e-01   4.79514897e-03]]
0.853156035477


In [22]:
model.save("stacked_unet.h5")